In [2]:
pip install onnxruntime-genai==0.7.0 --pre

Note: you may need to restart the kernel to use updated packages.


In [3]:
import onnxruntime_genai as og
import numpy as np
import os

In [4]:
model_folder = "./models/phi-4-mini/onnx/model"

In [5]:
model = og.Model(model_folder)

In [6]:
adapters = og.Adapters(model)
adapters.load('./models/phi-4-mini/onnx/model/adapter_weights.onnx_adapter', "qa_choice")

In [7]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

In [8]:
search_options = {}
search_options['max_length'] = 102
search_options['past_present_share_buffer'] = False

In [9]:
input = "Answer the following multiple-choice question by selecting the correct option.\n\nQuestion: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?\nAnswer Choices:\n(A) ignore\n(B) enforce\n(C) authoritarian\n(D) yell at\n(E) avoid"

In [10]:
chat_template = "</system>You are a helpful assistant. Your output should only be one of the five choices: 'A', 'B', 'C', 'D', or 'E'.<|end|><|user|>{input}<|end|><|assistant|>"

In [11]:
prompt = f'{chat_template.format(input=input)}'

In [12]:
input_tokens = tokenizer.encode(prompt)

In [13]:
params = og.GeneratorParams(model)
params.set_search_options(**search_options)
generator = og.Generator(model, params)

In [14]:
generator.set_active_adapter(adapters, "qa_choice")

In [15]:
generator.append_tokens(input_tokens)

In [16]:
while not generator.is_done():
            generator.generate_next_token()

            new_token = generator.get_next_tokens()[0]
            print(tokenizer_stream.decode(new_token), end='', flush=True)

A